In [10]:

import dill
import panel as pn
import json
import altair as alt

with open('gdf_merged.dill', 'rb') as f:
    gdf_merged = dill.load(f)

variable_list = ['Apartment Construction/Maintenance', 'Building Maintenance',
                 'Heat/Hot Water/Electricity', 'Noise - Outside',
                 'Noise - Residential', 'Parking/Street Conditions', 'Rodent',
                 'Unhealthy Conditions', 'Water Leaks and Plumbing']
variable = pn.widgets.Select(options=variable_list, name='Variable')


@pn.depends(variable.param.value)
def get_map(variable):
    gdf_merged_complaint = gdf_merged[gdf_merged["Complaint Type"] == variable]
    choro_json = json.loads(gdf_merged_complaint.to_json())
    choro_data = alt.Data(values=choro_json['features'])
    return gen_map(geodata=choro_data, color_column='properties.count_col_yr',
                   tooltip=['properties.zipcode:O','properties.count_col_yr:Q'],
                   color_scheme='redblue').repeat(row=[variable])
    
def gen_map(geodata, color_column, tooltip, color_scheme):
    
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(stroke='white').encode(
        alt.Color(color_column,type='quantitative',
                  scale=alt.Scale(scheme=color_scheme, reverse = True),title = "counts per year"),
        tooltip=tooltip)
    return choro

pn.Row(
    pn.Column('# Complaint Type Map', pn.panel(variable)),
    get_map
).show()

Launching server at http://localhost:60834
